# Invader Defender 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from scipy.optimize import linprog

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) 
gridSize = 6 
state_count = gridSize*gridSize

In [3]:
class Invader_Defender():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        
        # deterministic transition ?
        self.transition_prob = 1 
        
        # initialize defender and invader states
        self.new_state = [0, 0, 0, 0]
        self.new_defender_state = [0, 0]
        self.new_invader_state = [0, 0]
        
        # set territory state
        self.territory_state = [4, 4]

        # create a list of all possible states in the game
        self.game_state_list = []
        for defender_state in self.states:
            for invader_state in self.states:
                combined_states = defender_state + invader_state
                self.game_state_list.append(combined_states)
        
        # create 2 lists of states representing defender and invader victory
        self.defender_won = []
        self.invader_won = []
        
        # create states representing defender victory
        for defender_state in self.states:
            for invader_state in self.states:
                distance = np.linalg.norm(np.array(defender_state) - np.array(invader_state))
                # if the invader is not at territory and within the capture range of defender = defender won
                if invader_state != self.territory_state and distance <= np.sqrt(2):
                    combined_states = defender_state + invader_state
                    self.defender_won.append(combined_states)
           
        # create states representing invader victory
        for defender_state in self.states:
            distance = np.linalg.norm(np.array(defender_state) - np.array(self.territory_state))
            # if the invader is at territory, and outside of the defender's capture range = invader won
            if distance > np.sqrt(2):
                combined_states = defender_state + self.territory_state
                self.invader_won.append(combined_states)
    
    def possible_states(self):
        """
        A function that returns a list of all possible states in the game
        """
        return self.game_state_list
    
    def terminal_check(self, state):
        """
        A function that checks whether the game is at a terminal state.
        Terminal state happens when either the invader or defender has won.
        """
        if state in self.defender_won:
            status = "Defender Won"
            terminal_check = True
        elif state in self.invader_won:
            status = "Invader Won"
            terminal_check = True
        else:
            terminal_check = False
            status = "Game in Progress"

        return terminal_check, status
    
#     def transition_probability(self, transition):
#         """
#         A function that returns the transition probability...?
#         """
#         return self.transition_prob, reward

    def next_state(self, current_state, defender_action, invader_action):
        """
        A function that returns the next state
        Input: current state [0,0] , defender_action [0, 1], invader_action [0,-1]
        Output: next state array([x1,y1,x2,y2]) and reward (int)
            - If the action takes the agent off grid, the agent remains in original state
            - If defender won, reward is calculated based on manhattan distance between invader captured state
            and territory
            - If defender loss, reward is -100
        """
        defender_state = []
        invader_state = []
        
        # deconstruct current state [0,0,1,1] in to defender [0,0] and invader [1,1] state
        for i in range(4):
            if i < 2:
                defender_state.append(current_state[i])
            else:
                invader_state.append(current_state[i])
                
        # get next state: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_defender_state = list(np.array(defender_state) + np.array(defender_action))
        self.new_invader_state = list(np.array(invader_state) + np.array(invader_action))

        # if new defender states results in off the grid, return to original state
        if -1 in self.new_defender_state or self.size in self.new_defender_state:
            self.new_defender_state = defender_state
        
        # if new invader states results in off the grid, return to original state
        if -1 in self.new_invader_state or self.size in self.new_invader_state:
            self.new_invader_state = invader_state
       
        # combine the defender and invader state
        self.new_state = self.new_defender_state
        self.new_state.extend(self.new_invader_state)
        
        # calculate rewards
        terminal, status = self.terminal_check(self.new_state)
        if terminal == True:
            if status == "Defender Won":
                # defender reward if defender won (manhattan distance between invader captured state and territory)
                distance_to_territory = sum(abs(np.array(self.new_invader_state) - np.array(self.territory_state)))
                self.reward = distance_to_territory
            else:
                # defender reward if invader won
                self.reward = -100
        else:
            self.reward = 0
            
        return self.new_state, self.reward

## Testing 

In [4]:
invader_defender = Invader_Defender(6)

In [5]:
next_state, reward = invader_defender.next_state([2,1,0,0], [-1, 0], [-1, 0])

In [6]:
next_state

[1, 1, 0, 0]

In [7]:
reward

8

In [8]:
invader_defender.terminal_check([1, 1, 0, 0])

(True, 'Defender Won')

## Initialization and Function Definitions

In [9]:
invader_defender = Invader_Defender(6)

In [10]:
k = 0
U = {}
gamma = 0.9
state_list = []
listofzeros = [0.0] * len(invader_defender.game_state_list)
delta_list = []

# convert game_state_list in to a state list of tuples in order to make a dictionary
for state in invader_defender.game_state_list:
    state_list.append(tuple(state))
    
# initiate params
G = dict(zip(state_list, listofzeros))
U[k] = dict(zip(state_list, listofzeros))

In [12]:
def calculate_payoff(state):
    """
    A function calculates the payoff of a specific state by iterating over every defender/invader action
    Input: state (ie. [0,0,1,1])
    Output: payoff = 4x4 matrix where each element represent the defender's payoff 
    when defender take i, and invader take action j
    """
    payoff = np.zeros([4,4])
    for i in range(action_count):
        defender_action = actions[i]
        for j in range(action_count):
            invader_action = actions[j]
            next_state, reward = invader_defender.next_state(state, defender_action, invader_action)
            payoff[i, j] = reward + gamma*invader_defender.transition_prob*U[k][tuple(next_state)]
    return payoff

In [13]:
def calculate_value(G_state):
    """
    A function that calculates the value of a game by using linear programming.
    The value is calculated in both the defender and invader's perspective which are equal in value
    and opposite in signs
    Input: payoff matrix of a particular state (4x4 matrix)
    Output: Value = scalar value of the game.
    """
    
    # defender lin prog
    c = [0, 0, 0, 0, -1]
    defender_q = -1*np.transpose(G_state)     
    v_coeff = np.ones((4,1))
    Aub = np.concatenate((defender_q,v_coeff),1)
    b = [0, 0, 0, 0]
    Aeq = [[1, 1, 1, 1, 0]]
    beq = [[1.]]
    bounds = ((0,1),(0,1),(0,1),(0,1),(None, None))
    defender_solution = linprog(c, A_ub=Aub, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
    
    # invader lin prog
    c = [0, 0, 0, 0, 1]
    invader_q = G_state
    w_coeff = np.ones((4,1))*-1
    Aub = np.concatenate((invader_q,w_coeff),1)
    invader_solution = linprog(c, A_ub=Aub, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
    
    defender_value = defender_solution['fun']*-1
    invader_value = invader_solution['fun']*-1
    
    return defender_value, invader_value

In [14]:
def equilibrium(G_state):
    """
    A function that obtains the policy for defender and invader
    The value is calculated in both the defender and invader's perspective which are equal in value
    and opposite in signs
    Input: payoff matrix of a particular state (4x4 matrix)
    Output: policy for defender and invader
    """
    
    # defender lin prog
    c = [0, 0, 0, 0, -1]
    defender_q = -1*np.transpose(G_state)     
    v_coeff = np.ones((4,1))
    Aub = np.concatenate((defender_q,v_coeff),1)
    b = [0, 0, 0, 0]
    Aeq = [[1, 1, 1, 1, 0]]
    beq = [[1.]]
    bounds = ((0,1),(0,1),(0,1),(0,1),(None, None))
    defender_solution = linprog(c, A_ub=Aub, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
    
    # invader lin prog
    c = [0, 0, 0, 0, 1]
    invader_q = G_state
    w_coeff = np.ones((4,1))*-1
    Aub = np.concatenate((invader_q,w_coeff),1)
    invader_solution = linprog(c, A_ub=Aub, b_ub=b, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')
    
    defender_policy = defender_solution['x'][:4]
    invader_policy = invader_solution['x'][:4]
    
    return defender_policy, invader_policy

### Minmax Q-learning

In [15]:
# actions = [[-1, 0], [0, 1], [1, 0], [0, -1]]

# for the purpose of playing playing policy with some exploration
epsilon = 1

alpha = 0.5

gamma = 0.99

def action_explore(policy_action):
    # choose an action type: explore or exploit
    action_type = int(np.random.choice(2, 1, p=[epsilon,1-epsilon]))
    
    result = 0
    
    if action_type == 0:
        # explore
        random_action = np.random.choice(range(len(actions)))
        while random_action == policy_action:
            random_action = np.random.choice(range(len(actions)))
        result = random_action
        
    else:
        result = policy_action
    
    return result

Q_keys = []
for state in invader_defender.game_state_list:
    Q_keys.append(tuple(state))

listofzeros_Q = [np.zeros((4,4))] * len(Q_keys)
Q = dict(zip(Q_keys, listofzeros_Q))
#print(Q)

# Set s to current state
initial_state = (4, 1, 0, 0)
state = initial_state

# Build game G(s)
# G_state = Q[state].copy()
# print(G_state)

# Choose a policy pi_i(s) = Equilibrium_i(G(s))
defender_policy, invader_policy = equilibrium(Q[state])

T=500
t=1

while t <= T:
    # Play policy pi_i_s with some exploration
    defender_action = actions[action_explore(np.argmax(defender_policy))]
    invader_action = actions[action_explore(np.argmax(invader_policy))]
    #invader_action = actions[np.random.choice(range(len(actions)))]
    
    # Get new state and reward
    new_state, _ = invader_defender.next_state(list(state), defender_action, invader_action)
    new_state = tuple(new_state) # because it originally returns a list
    
    print(defender_action)
    print(invader_action)
    print(new_state)
    
    reward_matrix = np.zeros(tuple([len(actions)] * 2))
    for i in range(len(actions)):
        for j in range(len(actions)):
            _, reward = invader_defender.next_state(list(state), actions[i], actions[j])
            reward_matrix[i, j] = reward
    
    #print(reward_matrix)
    
    # Choose a policy pi_i_s' = eqbm_i_G_s'
    defender_policy, invader_policy = equilibrium(Q[new_state])
    
    # Formula 8.2
    value_next_state =  np.dot(np.dot(defender_policy, Q[new_state]), invader_policy)
    Q[state] = Q[state] + alpha * (reward_matrix + gamma * value_next_state - Q[state])
    
    is_terminal, _ = invader_defender.terminal_check(list(new_state))
    print(is_terminal)
    
    if is_terminal:
        state = initial_state
    else:
        state = new_state
        
    t+=1

final_defender_policy = {}
#print([key for key in Q.keys() if not invader_defender.terminal_check(list(key))])
for state in [key for key in Q.keys() if not invader_defender.terminal_check(list(key))[0]]:
    final_defender_policy[state], _ = equilibrium(Q[state])

from pprint import pprint
pprint(final_defender_policy)


[1, 0]
[0, 1]
(5, 1, 0, 1)
False
[1, 0]
[0, 1]
(5, 1, 0, 2)
False
[1, 0]
[1, 0]
(5, 1, 1, 2)
False
[0, 1]
[0, 1]
(5, 2, 1, 3)
False
[0, 1]
[0, 1]
(5, 3, 1, 4)
False
[1, 0]
[0, 1]
(5, 3, 1, 5)
False
[1, 0]
[1, 0]
(5, 3, 2, 5)
False
[-1, 0]
[-1, 0]
(4, 3, 1, 5)
False
[0, 1]
[1, 0]
(4, 4, 2, 5)
False
[-1, 0]
[-1, 0]
(3, 4, 1, 5)
False
[1, 0]
[0, 1]
(4, 4, 1, 5)
False
[0, 1]
[1, 0]
(4, 5, 2, 5)
False
[1, 0]
[-1, 0]
(5, 5, 1, 5)
False
[0, 1]
[-1, 0]
(5, 5, 0, 5)
False
[-1, 0]
[1, 0]
(4, 5, 1, 5)
False
[-1, 0]
[0, 1]
(3, 5, 1, 5)
False
[1, 0]
[0, 1]
(4, 5, 1, 5)
False
[1, 0]
[1, 0]
(5, 5, 2, 5)
False
[-1, 0]
[1, 0]
(4, 5, 3, 5)
True
[-1, 0]
[1, 0]
(3, 1, 1, 0)
False
[0, 1]
[0, 1]
(3, 2, 1, 1)
False
[0, 1]
[0, 1]
(3, 3, 1, 2)
False
[1, 0]
[1, 0]
(4, 3, 2, 2)
False
[0, 1]
[1, 0]
(4, 4, 3, 2)
False
[1, 0]
[1, 0]
(5, 4, 4, 2)
False
[-1, 0]
[1, 0]
(4, 4, 5, 2)
False
[1, 0]
[-1, 0]
(5, 4, 4, 2)
False
[-1, 0]
[1, 0]
(4, 4, 5, 2)
False
[0, 1]
[0, 1]
(4, 5, 5, 3)
False
[0, 1]
[0, 1]
(4, 5, 5, 4)
True

False
[0, 1]
[1, 0]
(5, 5, 3, 5)
False
[-1, 0]
[1, 0]
(4, 5, 4, 5)
True
[0, 1]
[1, 0]
(4, 2, 1, 0)
False
[1, 0]
[0, 1]
(5, 2, 1, 1)
False
[0, 1]
[0, 1]
(5, 3, 1, 2)
False
[1, 0]
[1, 0]
(5, 3, 2, 2)
False
[0, 1]
[0, 1]
(5, 4, 2, 3)
False
[-1, 0]
[1, 0]
(4, 4, 3, 3)
True
[-1, 0]
[1, 0]
(3, 1, 1, 0)
False
[-1, 0]
[1, 0]
(2, 1, 2, 0)
True
[0, 1]
[1, 0]
(4, 2, 1, 0)
False
[-1, 0]
[-1, 0]
(3, 2, 0, 0)
False
[1, 0]
[1, 0]
(4, 2, 1, 0)
False
[0, 1]
[-1, 0]
(4, 3, 0, 0)
False
[-1, 0]
[0, 1]
(3, 3, 0, 1)
False
[1, 0]
[1, 0]
(4, 3, 1, 1)
False
[0, 1]
[0, 1]
(4, 4, 1, 2)
False
[1, 0]
[-1, 0]
(5, 4, 0, 2)
False
[1, 0]
[-1, 0]
(5, 4, 0, 2)
False
[-1, 0]
[-1, 0]
(4, 4, 0, 2)
False
[0, 1]
[1, 0]
(4, 5, 1, 2)
False
[1, 0]
[-1, 0]
(5, 5, 0, 2)
False
[-1, 0]
[1, 0]
(4, 5, 1, 2)
False
[0, 1]
[-1, 0]
(4, 5, 0, 2)
False
[-1, 0]
[0, 1]
(3, 5, 0, 3)
False
[1, 0]
[0, 1]
(4, 5, 0, 4)
False
[-1, 0]
[1, 0]
(3, 5, 1, 4)
False
[0, 1]
[1, 0]
(3, 5, 2, 4)
True
[0, 1]
[0, 1]
(4, 2, 0, 1)
False
[0, 1]
[1, 0]
(4, 3, 1, 

{(0, 0, 0, 2): array([0., 0., 0., 1.]),
 (0, 0, 0, 3): array([0., 0., 0., 1.]),
 (0, 0, 0, 4): array([0., 0., 0., 1.]),
 (0, 0, 0, 5): array([0., 0., 0., 1.]),
 (0, 0, 1, 2): array([0., 0., 0., 1.]),
 (0, 0, 1, 3): array([0., 0., 0., 1.]),
 (0, 0, 1, 4): array([0., 0., 0., 1.]),
 (0, 0, 1, 5): array([0., 0., 0., 1.]),
 (0, 0, 2, 0): array([0., 0., 0., 1.]),
 (0, 0, 2, 1): array([0., 0., 0., 1.]),
 (0, 0, 2, 2): array([0., 0., 0., 1.]),
 (0, 0, 2, 3): array([0., 0., 0., 1.]),
 (0, 0, 2, 4): array([0., 0., 0., 1.]),
 (0, 0, 2, 5): array([0., 0., 0., 1.]),
 (0, 0, 3, 0): array([0., 0., 0., 1.]),
 (0, 0, 3, 1): array([0., 0., 0., 1.]),
 (0, 0, 3, 2): array([0., 0., 0., 1.]),
 (0, 0, 3, 3): array([0., 0., 0., 1.]),
 (0, 0, 3, 4): array([0., 0., 0., 1.]),
 (0, 0, 3, 5): array([0., 0., 0., 1.]),
 (0, 0, 4, 0): array([0., 0., 0., 1.]),
 (0, 0, 4, 1): array([0., 0., 0., 1.]),
 (0, 0, 4, 2): array([0., 0., 0., 1.]),
 (0, 0, 4, 3): array([0., 0., 0., 1.]),
 (0, 0, 4, 5): array([0., 0., 0., 1.]),


 (2, 4, 2, 2): array([0., 0., 0., 1.]),
 (2, 4, 3, 0): array([0., 0., 0., 1.]),
 (2, 4, 3, 1): array([0., 0., 0., 1.]),
 (2, 4, 3, 2): array([0., 0., 0., 1.]),
 (2, 4, 4, 0): array([0., 0., 0., 1.]),
 (2, 4, 4, 1): array([0., 0., 0., 1.]),
 (2, 4, 4, 2): array([0., 0., 0., 1.]),
 (2, 4, 4, 3): array([0., 0., 0., 1.]),
 (2, 4, 4, 5): array([0., 0., 0., 1.]),
 (2, 4, 5, 0): array([0., 0., 0., 1.]),
 (2, 4, 5, 1): array([0., 0., 0., 1.]),
 (2, 4, 5, 2): array([0., 0., 0., 1.]),
 (2, 4, 5, 3): array([0., 0., 0., 1.]),
 (2, 4, 5, 4): array([0., 0., 0., 1.]),
 (2, 4, 5, 5): array([0., 0., 0., 1.]),
 (2, 5, 0, 0): array([0., 0., 0., 1.]),
 (2, 5, 0, 1): array([0., 0., 0., 1.]),
 (2, 5, 0, 2): array([0., 0., 0., 1.]),
 (2, 5, 0, 3): array([0., 0., 0., 1.]),
 (2, 5, 0, 4): array([0., 0., 0., 1.]),
 (2, 5, 0, 5): array([0., 0., 0., 1.]),
 (2, 5, 1, 0): array([0., 0., 0., 1.]),
 (2, 5, 1, 1): array([0., 0., 0., 1.]),
 (2, 5, 1, 2): array([0., 0., 0., 1.]),
 (2, 5, 1, 3): array([0., 0., 0., 1.]),


 (5, 3, 1, 0): array([0., 0., 0., 1.]),
 (5, 3, 1, 1): array([0., 0., 0., 1.]),
 (5, 3, 1, 2): array([0., 0., 0., 1.]),
 (5, 3, 1, 3): array([0., 0., 0., 1.]),
 (5, 3, 1, 4): array([0., 0., 0., 1.]),
 (5, 3, 1, 5): array([0., 0., 0., 1.]),
 (5, 3, 2, 0): array([0., 0., 0., 1.]),
 (5, 3, 2, 1): array([0., 0., 0., 1.]),
 (5, 3, 2, 2): array([0., 0., 0., 1.]),
 (5, 3, 2, 3): array([0., 0., 0., 1.]),
 (5, 3, 2, 4): array([0., 0., 0., 1.]),
 (5, 3, 2, 5): array([0., 0., 0., 1.]),
 (5, 3, 3, 0): array([0., 0., 0., 1.]),
 (5, 3, 3, 1): array([0., 0., 0., 1.]),
 (5, 3, 3, 2): array([0., 0., 0., 1.]),
 (5, 3, 3, 3): array([0., 0., 0., 1.]),
 (5, 3, 3, 4): array([0., 0., 0., 1.]),
 (5, 3, 3, 5): array([0., 0., 0., 1.]),
 (5, 3, 4, 0): array([0., 0., 0., 1.]),
 (5, 3, 4, 1): array([0., 0., 0., 1.]),
 (5, 3, 4, 4): array([0., 0., 0., 1.]),
 (5, 3, 4, 5): array([0., 0., 0., 1.]),
 (5, 3, 5, 0): array([0., 0., 0., 1.]),
 (5, 3, 5, 1): array([0., 0., 0., 1.]),
 (5, 3, 5, 5): array([0., 0., 0., 1.]),


In [16]:
invader_defender.terminal_check([3,5,4,4])

(False, 'Game in Progress')

### Save Results to Pickle 

In [17]:
import pickle

# takes ~ 50 minutes (132 iterations) to converge to within tolerance, 
# so I am saving the learned U and G as a pickle
# to load them up faster (for development purpose)

with open('U.pickle', 'wb') as handle:
    pickle.dump(U, handle)

with open('G.pickle', 'wb') as handle:
    pickle.dump(G, handle)

In [18]:
# # uncomment to load U and G

# with open ('U.pickle', 'rb') as handle:
#     test_U = pickle.load(handle)
    
# with open ('G.pickle', 'rb') as handle:
#     test_G = pickle.load(handle)

# # converged k. This gives the last update to U dict
# k = 132

## Defender and Invader Policy 

In [21]:
defender_policy

array([0., 0., 0., 1.])

In [22]:
invader_policy

array([0., 0., 0., 1.])

## Shapley's Value Iteration 

In [ ]:
# to remove warnings
import warnings
warnings.filterwarnings('ignore')

tolerance = 1e-6
delta = 1
k = 0

while delta > tolerance:
   
    delta = 0
    
    # initialize the next entry of the U dictionary
    U[k+1] = dict(zip(state_list, listofzeros))

    for state in invader_defender.game_state_list:

        # Build G dictionary {state: payoff (4x4)}
        G[tuple(state)] = calculate_payoff(state)

        # calculate value of game
        defender_value, invader_value = calculate_value(G[tuple(state)])
#         print("state: ", state)
#         print("defender value: ", defender_value)
#         print("invader value: ", invader_value)
#         print("---------------------")

        # write value of game to the dictionary
        U[k+1][tuple(state)] = defender_value

        # calculate delta
        delta = max(delta, abs(U[k+1][tuple(state)]-U[k][tuple(state)]))
        
    # print k and current max delta
    clear_output(wait=True)
    display('k: ' + str(k) + ' delta: ' + str(delta))
    
    delta_list.append(delta)
    k += 1